In [1]:
import pandas as pd
from gensim.models import KeyedVectors
from nltk.corpus import stopwords
#nltk.download('stopwords')
from nltk.tokenize import word_tokenize

In [21]:
projetos = pd.read_csv('projetos_assembleia.csv')
print(len(projetos))
projetos.head()

32


,author,description,title,url
0,Luís Marques Mendes (PSD),Sobre a Agenda 2000,Projeto de Resolução 123/VII,https://www.parlamento.pt/ActividadeParlamenta...
1,"Luís Queiró (CDS-PP),Gonçalo Ribeiro da Costa ...",Conselhos municipais de acção social,Projeto de Lei 637/VII,https://www.parlamento.pt/ActividadeParlamenta...
2,"Pimenta Dias (PCP),Joaquim Matias (PCP),João A...",Isenta de emolumentos a apreciação das contas ...,Projeto de Lei 624/VII,https://www.parlamento.pt/ActividadeParlamenta...
3,"Luís Queiró (CDS-PP),Sílvio Rui Cervan (CDS-PP...","Pronuncia, acompanhamento e apreciação pela As...",Projeto de Lei 625/VII,https://www.parlamento.pt/ActividadeParlamenta...
4,Luís Queiró (CDS-PP),"Decreto-lei n° 15/99,de 15 de Janeiro, que""reg...",Apreciação Parlamentar 82/VII,https://www.parlamento.pt/ActividadeParlamenta...


In [3]:
# Carrega word2vec12'
model = KeyedVectors.load_word2vec_format("word2vec_200k.txt")

In [4]:
print(model.most_similar(positive=['saude']))
print(model.distance('saude', 'saude'))

[('reabilitação', 0.770971417427063), ('paternidade', 0.7249454855918884), ('orientação', 0.7113212943077087), ('preservação', 0.7078778147697449), ('regeneração', 0.7003017663955688), ('tutoria', 0.6979953646659851), ('preservaã§ã£o', 0.6955472230911255), ('mãe', 0.6948712468147278), ('limpeza', 0.6902233958244324), ('saúde', 0.6888123154640198)]
5.960464477539063e-08


In [22]:
def tokenize(text, remove_stop_words=True):
    text_tokens = word_tokenize(text)
    if(remove_stop_words):
        tokens_without_sw = [word for word in text_tokens if not word in stopwords.words()]
        return tokens_without_sw
    else:
        return text_tokens

for description in projetos['description']:
    tokens = tokenize(description)
    print(tokens)

['Sobre', 'Agenda', '2000']
['Conselhos', 'municipais', 'acção', 'social']
['Isenta', 'emolumentos', 'apreciação', 'contas', 'gerência', 'autarquias', 'locais']
['Pronuncia', ',', 'acompanhamento', 'apreciação', 'Assembleia', 'República', 'participação', 'Portugal', 'processo', 'construção', 'união', 'europeia']
['Decreto-lei', 'n°', '15/99', ',', '15', 'Janeiro', ',', "''", 'regula', 'intervenção', 'Estado', 'actividades', 'cinematográficas', 'audiovisual', 'multimédia', "''", '.']
['Define', 'princípios', 'referência', 'negociação', 'parte', 'portuguesa', 'Agenda', '2000', 'documentos', 'conexos']
['Altera', 'regime', 'instalação', 'novos', 'municípios', 'previsto', 'n°', '142/85', ',', '18', 'Novembro', ',', 'situação', 'ocorrência', 'eleições', 'prazo', 'curto']
['Elevação', 'categoria', 'vila', 'povoação', 'Santa', 'Luzia', 'município', 'Tavira']
['Elevação', 'povoação', 'Alcantarilha', 'concelho', 'Silves', 'categoria', 'vila']
['Criação', 'rede', 'publica', 'casas', 'apoio', 'mu

In [47]:
def max_AND_similarity(description_tokens, theme_tokens):
    max_similarities = []
    for theme_token in theme_tokens:
        max_similarity = -1.0
        for token in description_tokens:
            if(token in model and theme_token in model):
                new_similarity = model.similarity(token, theme_token)
                if (new_similarity > max_similarity):
                    max_similarity = new_similarity
        if(max_similarity < 0):
            max_similarity = 0.
        max_similarities.append(max_similarity)
    #print(max_similarities)
    prob = max_similarities[0]
    for sim in max_similarities[1:]:
        prob = prob * sim
    return prob

def max_similarity(description_tokens, theme_tokens):
    max_similarity = -1.0
    for token in description_tokens:
        for theme_token in theme_tokens:
            if(token in model and theme_token in model):
                new_similarity = model.similarity(token, theme_token)
                if (new_similarity > max_similarity):
                    max_similarity = new_similarity
    return max_similarity

def match_similarity(description_tokens, theme_tokens):
    found = 0
    for theme_token in theme_tokens:
        if(theme_token in description_tokens):
            found = found + 1
    if(found==len(theme_tokens)):
        return 1.
    else:
        return 0.

def find_projects(theme, limit=0.8, sim_type='max'):
    descriptions = projetos['description']
    authors = projetos['author']
    titles = projetos['title']
    out = []
    for i in range(len(titles)):
        description_tokens = tokenize(descriptions[i].lower())
        theme_tokens = tokenize(theme.lower())
        # Word Mover’s Distance
        #distance = model.wmdistance(description_tokens, theme_tokens)
        # max similarity
        similarity = 0.
        if(sim_type=='max'):
            similarity = max_similarity(description_tokens, theme_tokens)
        elif(sim_type=='max_and'):
            similarity = max_AND_similarity(description_tokens, theme_tokens)
        elif(sim_type=='exact_match'):
            similarity = match_similarity(description_tokens, theme_tokens)
        #print(descriptions[i], similarity)
        if(similarity>=limit):
            out.append([similarity, descriptions[i]])
    return out
        


In [48]:
atividades = find_projects('obras públicas', limit=0.9, sim_type='max')
for atividade in atividades:
    print("ATIVIDADE: {} | ADERÊNCIA: {:.02f}".format(atividade[1], atividade[0]))

print("----------------------------------------")

atividades = find_projects('obras públicas', sim_type='max_and')
for atividade in atividades:
    print("ATIVIDADE: {} | ADERÊNCIA: {:.02f}".format(atividade[1], atividade[0]))

print("----------------------------------------")

atividades = find_projects('obras públicas', sim_type='exact_match')
for atividade in atividades:
    print("ATIVIDADE: {} | ADERÊNCIA: {:.02f}".format(atividade[1], atividade[0]))


ATIVIDADE: Decreto-lei n° 15/99,de 15 de Janeiro, que"regula a intervenção do Estado nas actividades cinematográficas do audiovisual e do multimédia". | ADERÊNCIA: 0.91
ATIVIDADE: Decreto-lei n° 61/99 de 2 de Março que "define o acesso e permanência na actividade de empreiteiro de obras publicas e industrial de construção civil. Revoga o decreto-lei n°100/88 de 23 de Março. | ADERÊNCIA: 1.00
ATIVIDADE: Decreto-lei n° 59/99 de 2 de Março que "aprova o novo regime jurídico das empreitadas de obras públicas. | ADERÊNCIA: 1.00
----------------------------------------
ATIVIDADE: Decreto-lei n° 61/99 de 2 de Março que "define o acesso e permanência na actividade de empreiteiro de obras publicas e industrial de construção civil. Revoga o decreto-lei n°100/88 de 23 de Março. | ADERÊNCIA: 0.92
ATIVIDADE: Decreto-lei n° 59/99 de 2 de Março que "aprova o novo regime jurídico das empreitadas de obras públicas. | ADERÊNCIA: 1.00
----------------------------------------
ATIVIDADE: Decreto-lei n° 59/

In [49]:
atividades = find_projects('construção de obras públicas', limit=0.9, sim_type='max')
for atividade in atividades:
    print("ATIVIDADE: {} | ADERÊNCIA: {:.02f}".format(atividade[1], atividade[0]))

print("----------------------------------------")

atividades = find_projects('construção de obras públicas', sim_type='max_and')
for atividade in atividades:
    print("ATIVIDADE: {} | ADERÊNCIA: {:.02f}".format(atividade[1], atividade[0]))

print("----------------------------------------")

atividades = find_projects('construção de obras públicas', sim_type='exact_match')
for atividade in atividades:
    print("ATIVIDADE: {} | ADERÊNCIA: {:.02f}".format(atividade[1], atividade[0]))

ATIVIDADE: Pronuncia, acompanhamento e apreciação pela Assembleia da República da participação de Portugal no processo de construção da união europeia | ADERÊNCIA: 1.00
ATIVIDADE: Decreto-lei n° 15/99,de 15 de Janeiro, que"regula a intervenção do Estado nas actividades cinematográficas do audiovisual e do multimédia". | ADERÊNCIA: 0.91
ATIVIDADE: Altera o regime de instalação de novos municípios previsto na lei n° 142/85, de 18 de Novembro, para a situação de não ocorrência de eleições em prazo curto | ADERÊNCIA: 0.91
ATIVIDADE: Decreto-lei n° 61/99 de 2 de Março que "define o acesso e permanência na actividade de empreiteiro de obras publicas e industrial de construção civil. Revoga o decreto-lei n°100/88 de 23 de Março. | ADERÊNCIA: 1.00
ATIVIDADE: Decreto-lei n° 59/99 de 2 de Março que "aprova o novo regime jurídico das empreitadas de obras públicas. | ADERÊNCIA: 1.00
----------------------------------------
ATIVIDADE: Decreto-lei n° 61/99 de 2 de Março que "define o acesso e perman

In [50]:
atividades = find_projects('direito das mulheres', limit=0.9, sim_type='max')
for atividade in atividades:
    print("ATIVIDADE: {} | ADERÊNCIA: {:.02f}".format(atividade[1], atividade[0]))

print("----------------------------------------")

atividades = find_projects('direito das mulheres', sim_type='max_and')
for atividade in atividades:
    print("ATIVIDADE: {} | ADERÊNCIA: {:.02f}".format(atividade[1], atividade[0]))

print("----------------------------------------")

atividades = find_projects('direito das mulheres', sim_type='exact_match')
for atividade in atividades:
    print("ATIVIDADE: {} | ADERÊNCIA: {:.02f}".format(atividade[1], atividade[0]))

ATIVIDADE: Criação da rede publica de casas de apoio a mulheres vitimas de violência | ADERÊNCIA: 1.00
ATIVIDADE: Reforça as garantias do direito a saúde reprodutiva | ADERÊNCIA: 1.00
----------------------------------------
----------------------------------------


In [52]:
atividades = find_projects('corrupção', limit=0.7, sim_type='max')
for atividade in atividades:
    print("ATIVIDADE: {} | ADERÊNCIA: {:.02f}".format(atividade[1], atividade[0]))

print("----------------------------------------")

atividades = find_projects('corrupção', sim_type='max_and')
for atividade in atividades:
    print("ATIVIDADE: {} | ADERÊNCIA: {:.02f}".format(atividade[1], atividade[0]))

print("----------------------------------------")

atividades = find_projects('corrupção', sim_type='exact_match')
for atividade in atividades:
    print("ATIVIDADE: {} | ADERÊNCIA: {:.02f}".format(atividade[1], atividade[0]))

ATIVIDADE: Criação da rede publica de casas de apoio a mulheres vitimas de violência | ADERÊNCIA: 0.79
----------------------------------------
----------------------------------------
